## 2024 TBA Predictions

Match data is fetched from TBA by running fetchMatches.py.  Run this first before running this notebook.

`python fetchMatches.py`


In [1]:
from __future__ import print_function
import pickle
#import time
import sys
from collections import Counter
from tqdm import tqdm

sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException

# If you fetch_matches best to set reset=True or you may miss some events.
fetch_matches = False
reset = False
year = 2024

if fetch_matches:
    from fetchMatches import fetch_all_matches
    # This will save to matches_{year}.pkl
    fetch_all_matches(year, reset=reset)


In [2]:
# load all the matches

filename = f'matches_{year}.pkl'
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)

In [3]:
# [m for m in matches['events'] if m.address and 'Spokane' in m.address]
#[m for m in matches['events'] if 'pnc' in m.key]
pnw_district = [m.key for m in matches['events'] ] # if m.district and m.district.abbreviation=='pnw']
pnw_district

['2024alhu',
 '2024arli',
 '2024ausc',
 '2024azdd',
 '2024azgl',
 '2024azva',
 '2024bcvi',
 '2024brbr',
 '2024caav',
 '2024cabe',
 '2024cada',
 '2024cafr',
 '2024cala',
 '2024camb',
 '2024caoc',
 '2024caph',
 '2024casd',
 '2024casf',
 '2024casj',
 '2024cave',
 '2024chcmp',
 '2024cmptx',
 '2024cocs',
 '2024code',
 '2024cthar',
 '2024ctwat',
 '2024flor',
 '2024flta',
 '2024flwp',
 '2024gaalb',
 '2024gacar',
 '2024gacmp',
 '2024gadal',
 '2024gagwi',
 '2024hiho',
 '2024iacf',
 '2024idbo',
 '2024ilch',
 '2024ilpe',
 '2024incmp',
 '2024incol',
 '2024ineva',
 '2024inmis',
 '2024inpla',
 '2024iscmp',
 '2024isde1',
 '2024isde2',
 '2024isde3',
 '2024isde4',
 '2024ksla',
 '2024lake',
 '2024mabos',
 '2024mabri',
 '2024marea',
 '2024mawne',
 '2024mawor',
 '2024mdowi',
 '2024mdsev',
 '2024melew',
 '2024miann',
 '2024mibat',
 '2024mibel',
 '2024miber',
 '2024mibkn',
 '2024mibro',
 '2024micmp',
 '2024midet',
 '2024midtr',
 '2024miesc',
 '2024mike2',
 '2024miken',
 '2024miket',
 '2024mikk',
 '2024mikk2

Filter the matches to completed matches

In [4]:
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k]]
data = [m for m in data if m.winning_alliance!='' and m.score_breakdown is not None]
print(f'Found {len(data)} matches')

pnw_teams = set()
for m in [m for m in data if m.event_key in pnw_district]:
    for t in m.alliances.red.team_keys:
        pnw_teams.add(t)
    for t in m.alliances.blue.team_keys:
        pnw_teams.add(t)
    
pnw_teams = list(sorted(pnw_teams))
print(f'PNW Teams: {pnw_teams}')
#red = [x for m in data for x in m.alliances.red.team_keys]
#blue = [x for m in data for x in m.alliances.blue.team_keys]
#from collections import Counter
#Counter(red+blue)

Found 2725 matches
PNW Teams: ['frc1', 'frc100', 'frc1014', 'frc1023', 'frc1024', 'frc1058', 'frc1071', 'frc1076', 'frc108', 'frc1086', 'frc1089', 'frc11', 'frc1126', 'frc1143', 'frc1148', 'frc1153', 'frc1156', 'frc1159', 'frc1164', 'frc1165', 'frc1168', 'frc117', 'frc118', 'frc1197', 'frc1218', 'frc1247', 'frc1255', 'frc126', 'frc1262', 'frc1277', 'frc1279', 'frc1307', 'frc131', 'frc1318', 'frc1339', 'frc135', 'frc1351', 'frc1359', 'frc138', 'frc1391', 'frc1403', 'frc141', 'frc1410', 'frc144', 'frc1452', 'frc1466', 'frc1477', 'frc1501', 'frc1504', 'frc1506', 'frc1512', 'frc1522', 'frc1523', 'frc1528', 'frc1540', 'frc1555', 'frc1559', 'frc1569', 'frc157', 'frc1572', 'frc1574', 'frc1576', 'frc1577', 'frc1580', 'frc159', 'frc1592', 'frc1599', 'frc1619', 'frc1622', 'frc1629', 'frc1646', 'frc1657', 'frc166', 'frc1672', 'frc1676', 'frc1678', 'frc1690', 'frc1699', 'frc1701', 'frc1708', 'frc1714', 'frc1716', 'frc173', 'frc1731', 'frc1732', 'frc1740', 'frc1741', 'frc1744', 'frc1745', 'frc175',

In [5]:
pnw_matches = [m for m in data] # if m.event_key in pnw_district]
pnw_lookup = dict((k,i) for (i,k) in enumerate(pnw_teams))

In [6]:
# We'll only train based on qualifier matches
qualifiers = [x for x in data if x.comp_level=='qm'] 

Create aggregate team statistics for all teams, and a separate set for PNW teams

In [7]:
pnw_matches[0]
def get_rows(m):
    yield m.alliances.red.team_keys, m.score_breakdown['red']['totalPoints']
    yield m.alliances.blue.team_keys, m.score_breakdown['blue']['totalPoints']
list(get_rows(pnw_matches[0]))

[(['frc359', 'frc4270', 'frc6902'], 78),
 (['frc1622', 'frc216', 'frc4421'], 48)]

In [8]:
from scipy.sparse import csr_array
from scipy.sparse.linalg import spsolve, norm
import numpy as np

data = []
row = []
col = []
b = []
ctr = 0
for m in map(get_rows, pnw_matches):
    for r in m:
        for t in r[0]:
            row.append(len(b))
            col.append(pnw_lookup[t])
            data.append(1)
        b.append(r[1])
b = np.array(b)            
A = csr_array((data, (row, col)), shape=(len(b), len(pnw_lookup)))
print(A.shape, b.shape )
#x = spsolve(A, b)
#x
from scipy.sparse.linalg import lsqr
from scipy.linalg import lstsq

# Thanks ChatGPT!
x, residuals, rank, s = lstsq(A.todense(), b)

RSS = residuals.sum()
Rinv = np.linalg.inv(np.triu(s))

sigmas = np.sqrt(RSS / (len(b) - len(x)) * np.diag(Rinv))

#return_values = lsqr(A, b, calc_var=True)
#result = return_values
#print(result)
#x = return_values[0]
#var = return_values[-1]
#print(var)

opr = [(t,x[i],sigmas[i]) for i,t in enumerate(pnw_teams)]
for t,opr,sigma in sorted(opr, key=lambda x: x[1], reverse=True):
    print(t,opr,sigma)

#print((A@x).shape,b.shape)
print(A@x-b)
err = np.mean(A@x-b)
print(err)
opr_lookup = dict([(t,(x[i],sigmas[i])) for i,t in enumerate(pnw_teams)])


(5450, 1208) (5450,)
frc1678 55.3269336142196 5.516786326782624
frc1690 52.11699121717922 5.522143431383445
frc6329 47.65398485507451 7.069050383281285
frc359 45.56627844231047 6.499362697064601
frc3339 43.08112303072491 6.426741874912507
frc2468 42.28126395272863 5.93853887784546
frc1731 42.26251663092979 5.544035913209298
frc3005 41.90891346983617 6.227814625031987
frc6081 40.61792139983284 7.033212855959019
frc581 40.59643459737486 6.960763907171047
frc604 40.281074921172745 7.023492879847429
frc6036 39.933949981848166 7.022565724953133
frc1156 38.851884724014084 4.89249545058025
frc2910 38.783941849269226 6.194738571864674
frc9496 38.4923843405877 8.516315251774786
frc27 38.00033173777787 6.078120499788817
frc503 37.37616710517306 6.780079869965575
frc7763 36.84760787333372 7.371094165278522
frc4230 36.696122441191264 6.624371621112449
frc2231 36.29113542745091 5.848929164471825
frc179 36.05389813658688 5.601631357311813
frc3647 36.04394055702315 6.516499031649414
frc2075 35.639031

frc6217 14.837261084044592 7.052363383316821
frc2846 14.829362098705023 6.122743737551193
frc6107 14.815688079133654 7.035459450486636
frc3260 14.812903860131636 6.398155332053917
frc573 14.803450376353934 6.942319373532441
frc7289 14.799740486257559 7.273440491127903
frc9139 14.796934914973326 7.839253330874573
frc9567 14.795572316487547 8.779926465858786
frc8424 14.763712503454624 7.558191382712817
frc6502 14.750684440038851 7.11395668158797
frc9280 14.746968482450997 8.035931952958032
frc5253 14.72934325488398 6.827829586094609
frc2638 14.719987602376921 6.046819274062152
frc2926 14.699757065934122 6.200953736842436
frc3484 14.682249963977672 6.478853966742458
frc6647 14.636542710528548 7.1387177291301045
frc8839 14.63451050331993 7.660249883487828
frc6443 14.63234823636818 7.107096049956427
frc2813 14.63091578175062 6.104946943793735
frc930 14.41475888643575 8.082740197597511
frc3512 14.344358752149603 6.484862294679586
frc4206 14.328738993572735 6.608707858618831
frc9635 14.270930

frc9617 6.958617504083146 8.941279845198034
frc4501 6.951155199178169 6.68896392457301
frc4371 6.93384081261562 6.6547484485349715
frc5887 6.931507967917393 6.974938523469784
frc9203 6.916491539374329 7.94089730494413
frc5624 6.8892511102521885 6.907963681693437
frc7546 6.886130931188084 7.33454293897507
frc2998 6.840716207649712 6.2258596769316705
frc5872 6.828964549785807 6.974591705770136
frc5212 6.805845902412008 6.821343929042376
frc9239 6.79339281117961 8.018820213627684
frc6686 6.782883474852223 7.146108230287442
frc4627 6.743294131406219 6.713722200813026
frc9487 6.710354497134003 8.477200343426691
frc9504 6.693987938978364 8.52597144100577
frc5213 6.606694441567942 6.822449459761394
frc5746 6.58991704898031 6.951623202207313
frc238 6.579598094790583 5.90866268412905
frc8423 6.567316768377771 7.557078983203232
frc6872 6.5311957036802575 7.181997700088812
frc8020 6.526045491611739 7.428905912823228
frc5923 6.525336356565662 6.990970876986544
frc9121 6.5225733557842736 7.82066403

Next steps: for a given event we want to decide how to prioritize alliance choices.  Suppose we have all the data to date, as well as all the qualifier data for the event. Who should we choose as partners?

In [9]:

def predict(red,blue):
    mu = []
    sigma = []
    for r in red:
        mu.append(opr_lookup[r][0])
        sigma.append(opr_lookup[r][1])
    for b in blue:
        mu.append(-opr_lookup[b][0])
        sigma.append(opr_lookup[b][1])
    mu = sum(mu)
    sigma = np.linalg.norm(sigma)
    return(mu,sigma)

#predict(['frc2910'],['frc492'])
predict(['frc78'],['frc88'])


(7.562754262791536, 10.625598644399966)

In [10]:
## Brackets

alliances =  {
 'A1':   [7461, 2910,5827],
 'A2':  [488, 360, 4450],
 'A3':  [4911, 2412, 4512],
 'A4':  [5937, 1983, 3070],
 'A5':   [2976, 1899, 9023],
 'A6':   [492, 4682, 3681],
 'A7':   [1294, 948, 8248],
 'A8':   [9036, 949, 2928]
}

for k in alliances:
    alliances[k] = list(map(lambda x: f'frc{x}', alliances[k]))

bracket = {
    1: ['A1', 'A8'],
    2: ['A4', 'A5'],
    3: ['A2', 'A7'],
    4: ['A3', 'A6'],
    5: ['L1', 'L2'],
    6: ['L3', 'L4'],
    7: ['W1', 'W2'],
    8: ['W3', 'W4'],
    9: ['L7', 'W6'],
    10: ['W5', 'L8'],
    11: ['W7', 'W8'],
    12: ['W10', 'W9'],
    13: ['L11', 'W12'],
    14: ['W11', 'W13'],
    15: ['W14', 'L14'],
    16: ['W15', 'L15']
}

density = {i:Counter() for i in range(1,len(bracket)+1)}
        
def runMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    mu,sigma = predict(red,blue)
    r = np.random.normal(mu,sigma)
    #print(red,blue,mu,sigma,r)
    
    if r>0:        
        winner = red
        loser = blue
    else:
        winner = blue
        loser = red
    alliances[f'W{matchNumber}'] = winner
    alliances[f'L{matchNumber}'] = loser
    #print(f'{winner} beats {loser} by {abs(r)} in match {matchNumber}')

    
def pMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    return predict(red,blue)

import scipy.stats as stats

def pRed(matchNumber):
    mu,sigma = pMatch(matchNumber)
    return 1.0-stats.norm.cdf(0, loc=mu, scale=sigma)
    
    
def runBracket():
    for i in range(1,17):
        runMatch(i)        
    wins = Counter()
    for i in range(14,17):
        w = alliances[f'W{i}']
        wins[str(w)]+=1
    return sorted(wins, reverse=True, key=lambda x: wins[x])[0], (alliances['A6'] in [alliances['W11'],alliances['W13']])

overall = Counter()
inFinalCtr = 0
for b in tqdm(range(1000)):
    (w, inFinal) = runBracket()
    overall[w] += 1
    inFinalCtr += 1 if inFinal else 0
        
for k in sorted(overall, key=lambda x: overall[x], reverse=True):
    print(k, overall[k])

print(f'inFinal: {inFinalCtr}')

for k in sorted(density):
    print(k, density[k])

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


KeyError: 'frc7461'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import json
import seaborn as sns
colors = sns.color_palette(n_colors=8) # ('Dark2', n_colors=8)
cmap = matplotlib.colors.ListedColormap(colors)
plt.rcParams["figure.figsize"] = (20, 20)

allianceLookup = dict([(str(alliances[k]),k) for k in alliances if k[0]=='A'] )
alliance_list = list(allianceLookup.values())


def makelabel(s):
    #l = json.loads(s.replace('\'','"'))
    #return ','.join(map(lambda x: x[3:], l))
    return allianceLookup[s]
    
fig,axs= plt.subplots(len(density)//4, 4)
for a,i in enumerate(density):
    ax = axs[a//4,a%4]
    k = list(sorted(density[i], reverse=True, key=lambda d: density[i][d]))
    v = [density[i][d] for d in k]
    labels=list(map(makelabel, k))
    color_indices = [alliance_list.index(a) for a in labels]
    colors = [cmap(index) for index in color_indices]
    ax.pie(v, labels=labels, colors=colors, explode=[0.05]*len(labels)) #wedgeprops = {'linewidth': 3, 'linecolor':'white'})
    ax.set_title(f'Match {i}')

In [ ]:
# swap alliance 1 and 6 third choice

alliances =  {
 'A1':   [7461, 2910,3681],
 'A2':  [488, 360, 4450],
 'A3':  [4911, 2412, 4512],
 'A4':  [5937, 1983, 3070],
 'A5':   [2976, 1899, 9023],
 'A6':   [492, 4682, 5827],
 'A7':   [1294, 948, 8248],
 'A8':   [9036, 949, 2928]
}

for k in alliances:
    alliances[k] = list(map(lambda x: f'frc{x}', alliances[k]))


overall = Counter()
for b in tqdm(range(1000)):
    w = runBracket()
    overall[w] += 1
        
for k in sorted(overall, key=lambda x: overall[x], reverse=True):
    print(k, overall[k])